# Reference
- https://tensorboard-pytorch.readthedocs.io/en/latest/tutorial.html

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms
import torchvision.datasets as vdatasets
import torchvision.utils as vutils
import pickle
import os, shutil

torch.manual_seed(1)
import matplotlib.pyplot as plt
%matplotlib inline

# What is TensorBoardX?
- 구글의 tensorflow는 neural network의 훈련 과정을 시각화할 수 있는 tensorboard라는  웹서버가 있다. 이 tensorboard는 scalar, image, text 등을 정보를 시각화해준다. 하지만 pytorch같은 다른 딥러닝 프레임워크는 tensorboard와 같은 툴이 없다. 
- 그래서 tensorflow가 아닌 pytorch도 tensorboard의 기본적인 기능을 사용할 수 있게 해주는게 tensorboardX이다.
- 현재는 scalar, image, audio, histogram, text, embedding, route of backpropagation을 지원하고 있다.

# tensorboardX 사용법
## 0.port 설정

In [2]:
# 텐서보드 포트 설정
port= '6006'

# 텐서보드 데이터 파일 초기화
try:
    shutil.rmtree('runs/')
except:
    pass

In [3]:
port

'6006'

## 1. create a summary writer
-  logging을 하기 전에 writer instance를 생성해야함.

In [4]:
from tensorboardX import SummaryWriter

#SummaryWriter encapsulates everything
writer = SummaryWriter('runs/exp-1')
#creates writer object. The log will be saved in 'runs/exp-1'
writer2 = SummaryWriter()
#creates writer2 object with auto generated file name, the dir will be something like 'runs/Aug20-17-20-33'
writer3 = SummaryWriter(comment='3x learning rate')
#creates writer2 object with auto generated file name, the comment will be appended to the filename. The dir will be something like 'runs/Aug20-17-20-33-3xlearning rate'

## 2. add_
- logging할 정보를 입력한다.

### general api format
- add_something(tag_name, object, iteration number)

### 2.1 add_scalar

In [5]:
writer= SummaryWriter('runs/add_scalar')

In [6]:
for n_iter in range(100):
    s1= torch.rand(1)
    s2= torch.rand(1)
    writer.add_scalar('data/scalar1', s1[0], n_iter) # data grouping by 'slash'
    writer.add_scalars('data/scalar_group', {'xsinx':n_iter*np.sin(n_iter),
                                            'xcosx':n_iter*np.cos(n_iter),
                                            'arctan':np.arctan(n_iter)}, n_iter)
    

# Graph


In [7]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.linear1= nn.Linear(1024, 512)
        self.linear2= nn.Linear(512, 512)
        self.linear3= nn.Linear(512, 10)
        
        # In : {배치사이즈, 차원수} => Out : (배치사이즈, 차원수)
        self.bn1= nn.BatchNorm1d(512)
        self.bn2= nn.BatchNorm1d(512)
    def forward(self, inputs):
        outputs= self.bn1(self.linear1(inputs))
        outputs= F.relu(outputs)
        outputs= self.bn2(self.linear2(outputs))
        outputs= F.relu(outputs)
        return self.linear3(outputs)

In [8]:
model= NN()
test_inputs= torch.randn(64,1024, requires_grad=True)
outputs= model(test_inputs)

writer.add_graph(model, (test_inputs, ))  # , verbose=True)
writer.close()

In [19]:
# demo.py

import torch
import torchvision.utils as vutils
import numpy as np
import torchvision.models as models
from torchvision import datasets
from tensorboardX import SummaryWriter

resnet18 = models.resnet18(False)
writer= SummaryWriter(log_dir='resnet')
sample_rate = 44100
freqs = [262, 294, 330, 349, 392, 440, 440, 440, 440, 440, 440]


for n_iter in range(100):

    dummy_s1 = torch.rand(1)
    dummy_s2 = torch.rand(1)
    # data grouping by `slash`
    writer.add_scalar('data/scalar1', dummy_s1[0], n_iter)
    writer.add_scalar('data/scalar2', dummy_s2[0], n_iter)

    writer.add_scalars('data/scalar_group', {'xsinx': n_iter * np.sin(n_iter),
                                             'xcosx': n_iter * np.cos(n_iter),
                                             'arctanx': np.arctan(n_iter)}, n_iter)

    dummy_img = torch.rand(32, 3, 64, 64)  # output from network
    if n_iter % 10 == 0:
        x = vutils.make_grid(dummy_img, normalize=True, scale_each=True)
        writer.add_image('Image', x, n_iter)

        dummy_audio = torch.zeros(sample_rate * 2)
        for i in range(x.size(0)):
            # amplitude of sound should in [-1, 1]
            dummy_audio[i] = np.cos(freqs[n_iter // 10] * np.pi * float(i) / float(sample_rate))
        writer.add_audio('myAudio', dummy_audio, n_iter, sample_rate=sample_rate)

        writer.add_text('Text', 'text logged at step:' + str(n_iter), n_iter)

        for name, param in resnet18.named_parameters():
            writer.add_histogram(name, param.clone().cpu().data.numpy(), n_iter)

        # needs tensorboard 0.4RC or later
        writer.add_pr_curve('xoxo', np.random.randint(2, size=100), np.random.rand(100), n_iter)

dataset = datasets.MNIST('mnist', train=False, download=True)
images = dataset.test_data[:100].float()
label = dataset.test_labels[:100]

features = images.view(100, 784)
writer.add_embedding(features, metadata=label, label_img=images.unsqueeze(1))

# export scalar data to JSON for external processing
writer.export_scalars_to_json("./all_scalars.json")

writer.close()

tensorboard --logdir runs --port 6006